In [1]:
#!pip install python-igraph

# Load the graphs

In [1]:
#@title  { form-width: "30%" }
import tqdm as tq
import igraph as ig 
import os

datapath = "/Volumes/GoogleDrive/Il Mio Drive/TUDatasets" #@param {type:"string"}
dataset = "Kidney_9.2" #@param ["Mutag", "LFR", "MREG", "Kidney_9.2", "COBREpos"] {allow-input: true}
format = "graphml" #@param ["graphml", "edgelist"] {allow-input: true}
path = f'{datapath}/{dataset}/{format}'
filenames = os.listdir(path)
graphs = []
for f in tq.tqdm(filenames):
  if f.endswith(format):
    g = ig.load(os.path.join(path,f))
    g["name"] = f.split('.')[0]
    graphs += [g]

100%|██████████| 299/299 [00:04<00:00, 71.09it/s]


In [61]:
graphs[0].vs[1]['id']

'1'

# Line Graph transformers

In [149]:
def add_vertex_with_attrs(graph, attrs):
    n = graph.vcount()
    graph.add_vertices(1)
    for key, value in attrs.items():
        graph.vs[n][key] = value

def linegraph(g, nodeattr='name',debug=False):
    if nodeattr in g.attributes():
      raise ValueError(f'"{nodeattr}" is already a graph attribute... choose another one to store vertex names')
    dg = ig.Graph(directed=True) if g.is_directed() else ig.Graph() 
    for edge in g.es:
        edge_attrs = edge.attributes()
        edge_attrs[nodeattr] = set([int(edge.source),int(edge.target)]) # node name is the set of edge bounds
        if debug: print(edge_attrs)
        add_vertex_with_attrs(dg, edge_attrs)
    nodenames = dg.vs.get_attribute_values(nodeattr)
    for i in range(dg.vcount()):
      name = nodenames[i]
      for n in nodenames:
        common = name.intersection(n)
        if len(common) > 0 and n != name:
          nodeobj = dg.vs.find(**{nodeattr : name})
          nobj = dg.vs.find(**{nodeattr : n})
          if debug: print(n,name,common,{nodeattr : name},{nodeattr : n}, nodeobj.index, nobj.index)
          eattrvals = g.vs[common.pop()].attributes()
          if not dg.are_connected(nodeobj, nobj): dg.add_edge(nodeobj,nobj, **eattrvals) 
    return dg

In [151]:
linegraphs = []
for g in tq.tqdm(graphs):
    linegraphs += [linegraph(g, nodeattr='id')]

100%|██████████| 1/1 [02:22<00:00, 142.45s/it]


# Save line graphs (in graphml format)

In [ ]:
for i,g in enumerate(tq.tqdm(linegraphs)):
    g.write_graphml(f'../../GraphML/datasets/YENDIK/graphml/YENDIK_{i+1}.grahml')